Image based Fish Detection

In [ ]:
import cv2
import numpy as np

# Paths to your YOLO files
weights_path = "/Users/manu/Master/FishDetection/merge_yolo-fish-1.weights"
config_path = "/Users/manu/Master/FishDetection/yolo-fish-1.cfg"
image_path = "/Users/manu/Master/FishDetection/Data/cropped/A73EGS-P_3.png"  # Change to your test image
#video_path = "/Users/manu/Python/YoutubePremium/Data/Clownfish in Nature.mp4"

# Load YOLO model
net = cv2.dnn.readNet(weights_path, config_path)

# Load image
img = cv2.imread(image_path)
height, width = img.shape[:2]

# Prepare input blob
blob = cv2.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)

# Get output layer names
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

# Run forward pass
outs = net.forward(output_layers)

# Detection parameters
conf_threshold = 0.5
nms_threshold = 0.4

boxes = []
confidences = []
class_ids = []

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > conf_threshold:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Apply Non-Max Suppression
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

# Draw bounding boxes
for i in indices.flatten():
    x, y, w, h = boxes[i]
    label = f"Fish {class_ids[i]}"
    confidence = confidences[i]
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(img, f"{label}: {confidence:.2f}", (x, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

# Show result
cv2.imshow("Fish Detection", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Video based Fish Detection

In [ ]:
import cv2
import numpy as np

# Paths to your YOLO files
weights_path = "/Users/manu/Master/FishDetection/merge_yolo-fish-1.weights"
config_path = "/Users/manu/Master/FishDetection/yolo-fish-1.cfg"
# MODIFIED: Path to the video file
video_path = "/Users/manu/Downloads/Carp Fish Goldfish Angelfish Guppy Guppies Catfish animals Videos (1).mp4"

# Load YOLO model (this is done only once)
net = cv2.dnn.readNet(weights_path, config_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

# MODIFIED: Initialize video capture
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# MODIFIED: Loop through each frame of the video
while True:
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break  # Exit the loop if the video has ended

    height, width = frame.shape[:2]

    # Prepare input blob for the frame
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Run forward pass
    outs = net.forward(output_layers)

    # --- Post-processing for the current frame ---
    conf_threshold = 0.5
    nms_threshold = 0.4
    boxes = []
    confidences = []
    class_ids = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-Max Suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    # Draw bounding boxes on the frame
    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i]
            label = "Fish" # Simplified label
            confidence = confidences[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"{label}: {confidence:.2f}", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # MODIFIED: Show the processed frame
    cv2.imshow("Fish Detection", frame)

    # MODIFIED: Wait for a key press and exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# MODIFIED: Release resources
cap.release()
cv2.destroyAllWindows()

: 